In [1]:
import pandas as pd
import os
import time
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('gene2pubmed.gz', sep='\t') #20200103 downloaded

In [3]:
print(len(df))
df.head()

13035707


,#tax_id,GeneID,PubMed_ID
0,33,5961931,18245244
1,33,5961932,18245244
2,33,5961933,18245244
3,33,5961934,18245244
4,33,5961935,18245244


In [4]:
df_hs =df.loc[df['#tax_id']==9606,] 
print(len(df_hs))
df_hs.head()

1567493


,#tax_id,GeneID,PubMed_ID
2970546,9606,1,2591067
2970547,9606,1,3458201
2970548,9606,1,3610142
2970549,9606,1,8889549
2970550,9606,1,12477932


In [5]:
gene_l = df_hs.GeneID.unique().tolist()
print(len(gene_l))
print(len(df_hs))
gene_l[:5]

38738
1567493


[1, 2, 3, 9, 10]

In [6]:
# i = 1246500
df2 = pd.DataFrame()
df3 = pd.DataFrame()
data = {}

for i in tqdm(gene_l):
    i_l = df_hs.loc[df.GeneID==i,].PubMed_ID.tolist()
    data['i_l'] = i_l
    df3['GeneID'] = [i]
    df3['Pubmedcnt'] = [len(i_l)]
    df2 = pd.concat([df2, df3])
    

  0%|          | 0/38738 [00:00<?, ?it/s]

In [7]:
df2['rnk'] = df2['Pubmedcnt'].rank(ascending=False, method='min') 
df2.head()

,GeneID,Pubmedcnt,rnk
0,1,31,9952.0
0,2,272,921.0
0,3,6,19233.0
0,9,245,1059.0
0,10,704,228.0


In [8]:
humanGeneID = pd.read_csv('HumanGene_info.gz', sep='\t') #tax_id: 9606 in gene_info.gz
humanGeneID.head()

,#tax_id,GeneID,Symbol,Synonyms,description,Nomenclature_status
0,9606,1,A1BG,A1B|ABG|GAB|HYST2477,alpha-1-B glycoprotein,O
1,9606,2,A2M,A2MD|CPAMD5|FWP007|S863-7,alpha-2-macroglobulin,O
2,9606,3,A2MP1,A2MP,alpha-2-macroglobulin pseudogene 1,O
3,9606,9,NAT1,AAC1|MNAT|NAT-1|NATI,N-acetyltransferase 1,O
4,9606,10,NAT2,AAC2|NAT-2|PNAT,N-acetyltransferase 2,O


In [9]:
pubmedcnt = pd.merge(df2, humanGeneID[['GeneID', 'Symbol', 'description']], on='GeneID').rename(columns={'jaccard': 'ARNT_jacc', 'simpson': 'ARNT_simp', 'simp_rnk':'ARNT_simprnk'})
pubmedcnt.head()

,GeneID,Pubmedcnt,rnk,Symbol,description
0,1,31,9952.0,A1BG,alpha-1-B glycoprotein
1,2,272,921.0,A2M,alpha-2-macroglobulin
2,3,6,19233.0,A2MP1,alpha-2-macroglobulin pseudogene 1
3,9,245,1059.0,NAT1,N-acetyltransferase 1
4,10,704,228.0,NAT2,N-acetyltransferase 2


In [10]:
pubmedcnt_s = pubmedcnt.sort_values('Pubmedcnt', ascending = False)
pubmedcnt_s.head(10)

,GeneID,Pubmedcnt,rnk,Symbol,description
5607,7157,10084,1.0,TP53,tumor protein p53
5579,7124,5917,2.0,TNF,tumor necrosis factor
1549,1956,5806,3.0,EGFR,epidermal growth factor receptor
5826,7422,4706,4.0,VEGFA,vascular endothelial growth factor A
2777,3569,4672,5.0,IL6,interleukin 6
291,348,4489,6.0,APOE,apolipoprotein E
5500,7040,4422,7.0,TGFB1,transforming growth factor beta 1
3447,4524,3568,8.0,MTHFR,methylenetetrahydrofolate reductase
1664,2099,3291,9.0,ESR1,estrogen receptor 1
173,207,3279,10.0,AKT1,AKT serine/threonine kinase 1


In [11]:
pubmedcnt.to_csv("G2P_cnt.csv", index=False)